In [ ]:
# default_exp helper_functions

# helper_functions

> read and preprocess netcdf data

In [ ]:
#export
import xarray as xr
import numpy as np

In [ ]:
#export
def rotated_grid_transform(grid_in, option, SP_coor):
    """
    rotated pole
    """
    
    pi = np.pi
    lon = grid_in[0]
    lat = grid_in[1];

    lon = (lon*pi)/180; # Convert degrees to radians
    lat = (lat*pi)/180;

    SP_lon = SP_coor[0];
    SP_lat = SP_coor[1];
    
    SP_lon = SP_lon - 180
    SP_lat = -SP_lat

    theta = 90+SP_lat; # Rotation around y-axis
    phi = SP_lon; # Rotation around z-axis

    theta = (theta*pi)/180;
    phi = (phi*pi)/180; # Convert degrees to radians

    x = np.cos(lon)*np.cos(lat); # Convert from spherical to cartesian coordinates
    y = np.sin(lon)*np.cos(lat);
    z = np.sin(lat);

    if option == 1: # Regular -> Rotated

        x_new = np.cos(theta)*np.cos(phi)*x + np.cos(theta)*np.sin(phi)*y + np.sin(theta)*z;
        y_new = -np.sin(phi)*x + np.cos(phi)*y;
        z_new = -np.sin(theta)*np.cos(phi)*x - np.sin(theta)*np.sin(phi)*y + np.cos(theta)*z;

    else:  # Rotated -> Regular

        phi = -phi;
        theta = -theta;

        x_new = np.cos(theta)*np.cos(phi)*x + np.sin(phi)*y + np.sin(theta)*np.cos(phi)*z;
        y_new = -np.cos(theta)*np.sin(phi)*x + np.cos(phi)*y - np.sin(theta)*np.sin(phi)*z;
        z_new = -np.sin(theta)*x + np.cos(theta)*z;



    lon_new = np.arctan2(y_new,x_new); # Convert cartesian back to spherical coordinates
    lat_new = np.arcsin(z_new);

    lon_new = (lon_new*180)/pi; # Convert radians back to degrees
    lat_new = (lat_new*180)/pi;

    return lon_new , lat_new

def transform_calendar(ds,
                       timedim="time",
                       calendarname="proleptic_gregorin"):
    """Changes attributes of xarray dataset. Use in 
    combination with xr.open_dataset([...], decode_time=False)
    Args:
        ds (object): xarray dataset
    Kwargs:
        timedim (string): name of time dim
        calendarname (string): calendar name
    Returns: ds
    """
    ds[timedim].attrs['calendar'] = calendarname
    return ds

def read_netcdfs(files, dim, transform_func, transform_calendar=None):
    """Reads multiples netcdfs files. Should be used when open_mfdatasets
    is to slow.
    Parameters
    ----------
    files : Path to files (str)
    dim : dimension to concat files (if transform_calendar=T, concat along time)
    transform_func : additional preprocessing option
    transform_calendar : name of calendar (see function transform_calendar())
    Returns
    -------
    combined netcdf files as xarray object
    
    Example
    -------
    combined =  read_netcdfs("/silod5/boergel/tmp.nc", dim = "TIME",
                transform_func=lambda ds:ds.salt,
                transform_calendar="proleptic_gregorian")
    """
    def process_one_path(path):
        if transform_calendar is not None:
            calendar = False
        else:
            calendar = True
        with xr.open_dataset(path, decode_times = calendar) as ds:
            if transform_calendar is not None:
                ds[dim].attrs['calendar'] = transform_calendar
                ds = xr.decode_cf(ds)
            if transform_func is not None:
                ds = transform_func(ds)
            ds.load()
            return ds
    paths = sorted(glob(files))
    datasets = [process_one_path(p) for p in paths]
    combined = xr.concat(datasets, dim)
    return combined
        
def adjust_lon_lat(ds, lon_name, lat_name, reverse = False):
    """Adjusts longitude from 0 to 360 to -180 to 180
    and reverses lattiude.
    Args:
        ds (xarray): xarray Dataset
        lon_name (str): name of Longitude in ds
        lat_name (str): name of latitude in ds 
    Returns: ds (xarray)  
    """
    # Reverse lat
    if reverse == True:
        ds = ds.reindex({lat_name:ds[lat_name][::-1]})

    # Adjust longitude
    ds['_longitude_adjusted'] = xr.where(
        ds[lon_name] > 180,
        ds[lon_name] - 360,
        ds[lon_name])

    ds = (ds
          .swap_dims({lon_name: '_longitude_adjusted'})
          .sel(**{'_longitude_adjusted': sorted(ds._longitude_adjusted)})
          .drop(lon_name))

    ds = ds.rename({'_longitude_adjusted': lon_name})

    return ds